In [ ]:
pip install surprise

In [ ]:
import gzip
import random
import scipy
import tensorflow as tf
from collections import defaultdict
from implicit import bpr
from surprise import SVDpp, Reader, Dataset
from surprise.model_selection import train_test_split
import csv
import pandas as pd

In [3]:
trainData = []
file_path = './archive/interactions_train.csv'

with open(file_path, 'r') as file:
    reader = csv.reader(file)
    count = 0
    for row in reader:
        count +=1
        if count == 1:
            continue
        trainData.append([row[0], row[1], int(row[3][0])])
        
validData = []
file_path = './archive/interactions_validation.csv'

with open(file_path, 'r') as file:
    reader = csv.reader(file)
    count = 0
    for row in reader:
        count +=1
        if count == 1:
            continue
        validData.append([row[0], row[1], int(row[3][0])])
        
testData = []
file_path = './archive/interactions_test.csv'

with open(file_path, 'r') as file:
    reader = csv.reader(file)
    count = 0
    for row in reader:
        count +=1
        if count == 1:
            continue
        testData.append([row[0], row[1], int(row[3][0])])

In [ ]:
reader = Reader(rating_scale=(0, 5))

df_train = pd.DataFrame(trainData, columns=['userID', 'itemID', 'rating'])
data_train = Dataset.load_from_df(df_train[['userID', 'itemID', 'rating']], reader)

df_valid = pd.DataFrame(validData, columns=['userID', 'itemID', 'rating'])
data_valid = Dataset.load_from_df(df_valid[['userID', 'itemID', 'rating']], reader)

trainset = data_train.build_full_trainset()

# model = SVDpp(n_factors= 150, lr_all=0.003, reg_bu=0.4, reg_bi= 0.003, reg_pu= 0.47, reg_qi= 0.4,reg_yj=0.02, n_epochs= 75)

model = SVDpp(n_factors = 5)

model.fit(trainset)

predictions = model.test(testData)

#----------------------------------------------------------------#

# calculate mse
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print("MSE of initial latent factor model: " + str(sse / len(predictions)))

In [35]:
# calculate mse
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print("MSE of initial latent factor model: " + str(sse / len(predictions)))

MSE of latent factor model: 1.7291665320452776


In [36]:
print("actual rating: " + str(testData[0]))
print("predicted rating: " + str(predictions[0].est))

actual rating: ['8937', '44551', 4]
predicted rating: 4.24021703076739


In [60]:
# printing latent factors for first user and item
print(model.pu[0])
print(model.qi[0])

[-0.00517627  0.02802388 -0.01451376 -0.00522544 -0.01051014]
[ 0.11941598 -0.08573039 -0.09161672  0.12242995  0.06544003]


In [61]:
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'n_factors': [5],
              'lr_all': [0.004, 0.005, 0.006],
              'reg_all': [0.2, 0.25, 0.3],
              'n_epochs': [20]}

# Instantiate the SVDpp algorithm
svdpp = SVDpp

# Perform grid search with cross-validation
grid_search = GridSearchCV(svdpp, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data_train)

# Print the best hyperparameters and corresponding RMSE
print("Best Hyperparameters:", grid_search.best_params['rmse'])

# Train the model with the best hyperparameters on the full training set
best_svdpp = grid_search.best_estimator['rmse']
best_svdpp.fit(data_train.build_full_trainset())

# Evaluate on the validation set
valid_predictions = best_svdpp.test(data_valid.build_full_trainset().build_testset())
rmse_valid = accuracy.rmse(valid_predictions)
print("RMSE on Validation Set:", rmse_valid)

# Test the model on the test set
test_predictions = best_svdpp.test(testData)

# Calculate MSE on the test set
sse_test = sum([(p.r_ui - p.est)**2 for p in test_predictions])
mse_test = sse_test / len(test_predictions)
print("MSE on Test Set:", mse_test)

Best Hyperparameters: {'n_factors': 5, 'lr_all': 0.004, 'reg_all': 0.2, 'n_epochs': 20}
RMSE: 1.2804
RMSE on Validation Set: 1.280365563696972
MSE on Test Set: 1.7612393380092546
